In [1]:
import datetime
import time
from pandas_datareader import data as pdr
import time
import json
import pickle
import pandas as pd
import pickle

## Note:
- è giusto che ipos contenga anche la data per essere sicuri. altrimenti potresti prendere giorni precedenti a ipo
- bisogna fare webcrawling per ottenere la lista di tutte le ipo di nasdaq e nyse. Facendo cherry picking di ipos "celebri" a posteriori rischi di selezionare solo quelli che sono andati bene
- sarebbe da fare per vari anni (2021, 2020, 2016 ecc) per valutare varie condizioni di mercato

In [2]:
delta=20

ipos={}

results={"ticker":[],
         "volume":[],
         "max-gain":[],
         "max-loss":[]}

path_in="data/mar2021.txt"
path_out_excel=path_in.replace(".txt",".xlsx")
path_out_pkl=path_in.replace(".txt",".pkl")

i=0
with open(path_in,"r",encoding="utf-8") as r:
    for line in r:
        line=line.strip()
        if i%8==0:
            ticker=line
        if i%8==5:
            line_date=line.split("/")
            try:
                date=(int(line_date[-1]),int(line_date[0]),int(line_date[1]))
            except:
                print("@@@",i)
            ipos[ticker]=date
        i+=1

In [3]:
print(len(ipos))

151


In [4]:
for ticker,date in ipos.items():
    start_date=datetime.datetime(date[0],date[1],date[2])
    end_date=start_date+datetime.timedelta(days=delta)
    try:
        df = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
        df.drop("Adj Close",axis=1,inplace=True)
        print("### ",ticker)
        # print(df.head())
        # print()
        max_gain=(max(df["High"])-df["Close"][0])/df["Close"][0]*100
        max_loss=(min(df["Low"])-df["Close"][0])/df["Close"][0]*100
        results["ticker"].append(ticker)
        results["volume"].append(df["Volume"][0])
        results["max-gain"].append(round(max_gain,1))
        results["max-loss"].append(round(max_loss,1))
        time.sleep(2)
    except:
        print("No data for ticker ",ticker)

###  COUR
###  ACHL
###  MBTCU
###  TWOA
###  HYW
###  EWTX
###  DSGN
###  TDUP
###  ZH
###  IKNA
###  ALHC
###  VZIO
###  SEMR
###  OLK
###  MSDAU
###  CRCT
###  EEIQ
###  DSEY
###  LCAHU
###  WMPN
###  LVTX
###  KVSC
###  KVSB
###  DOCN
No data for ticker  NGC'U
###  ACVA
###  DISAU
###  DCRCU
###  CRZNU
###  NGCAU
###  NBSTU
###  DGNU
###  MOVE
###  LEGAU
###  UPC
No data for ticker  RKTAU
No data for ticker  VGIIU
No data for ticker  OPA'U
No data for ticker  STREU
###  ACTDU
###  GLHAU
###  GGPIU
###  TBSAU
###  GGMCU
###  FNCH
No data for ticker  LGV'U
###  TIL
###  AFCG
###  AFAQU
###  LDHAU
###  LVRAU
###  BYTSU
###  CNTB
###  NAPA
###  REVHU
###  RACB
###  VEI
###  GXIIU
No data for ticker  AAQCU
###  AMAOU
###  FMIVU
###  TUYA
###  GANX
###  KSICU
###  JWEL
###  GAMCU
###  TETCU
No data for ticker  KAHCU
No data for ticker  BGSXU
No data for ticker  ATHNU
###  SNCY
###  OLO
No data for ticker  FVIVU
###  RTPYU
###  PLMIU
###  EBACU
###  WALDU
No data for ticker  FZT'U
###  FT

In [5]:
df_res=pd.DataFrame(results)
print(df_res)

    ticker    volume  max-gain  max-loss
0     COUR  10361300      39.0     -16.0
1     ACHL   2110800      14.5     -23.7
2    MBTCU    254400       4.3       0.0
3     TWOA   6129100       1.8      -1.0
4      HYW   2134000      13.6     -30.8
..     ...       ...       ...       ...
108  DHBCU   8187100       1.5      -1.4
109  NDACU   9806200       5.7      -1.3
110  OHPAU   9258800       8.3      -1.3
111  ARRWU   8613900       1.9      -0.8
112  ANZUU  20111200       1.0      -0.8

[113 rows x 4 columns]


In [6]:
df_res.to_excel(path_out_excel)
pickle.dump(df_res, open(path_out_pkl, 'wb'))